In [11]:
import time                      # import libraries
start_time = time.time()

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import imblearn
from imblearn.over_sampling import SMOTE
import collections
from tqdm import tqdm_notebook as tqdm

from sklearn.preprocessing import StandardScaler      # import sklearn libraries
from sklearn import datasets, model_selection
from sklearn.metrics import plot_confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix,plot_roc_curve
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.naive_bayes import GaussianNB

from xgboost import XGBClassifier                     # import non-sklearn algorithm libraries
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier


In [12]:
print("--- %s seconds ---" % round((time.time() - start_time),0))
df = pd.read_csv('./DATA/creditcard_train1.csv')
df2 = pd.read_csv('./DATA/creditcard_train2.csv')
df = pd.concat([df,df2[1:]])
del df2

--- 1.0 seconds ---
